In [95]:
import os
import numpy as np
import pandas as pd

from datetime import datetime, date, timedelta

import warnings
from termcolor import colored

# from db.bin.indicators import lowpass_filter
from scipy.signal import butter,lfilter

from statsmodels.tsa.arima.model import ARIMA
import arch

from tqdm import tqdm
from pathlib import Path

In [2]:
DATA_PATH = os.path.join(os.path.abspath('../../'), 'db', 'data')
TERTIARY_PATH = os.path.join(DATA_PATH, 'tertiary')
wpath = os.path.join(DATA_PATH, 'merge')

In [67]:
# Curency indexes dataframe
raw = pd.read_csv(os.path.join(wpath, os.path.basename(TERTIARY_PATH), 'logs_.csv'), parse_dates=['Unnamed: 0'], index_col='Unnamed: 0')
raw.index = pd.to_datetime(raw.index.values)
print(raw.tail())

# today = datetime.now()
# hour = datetime( today.year, today.month, today.day, today.hour)
# today = date.today()
# today
max = datetime(2022, 8, 1)
max = pd.Timestamp(max,tz='UTC')
max

                                AUD       CAD       CHF       EUR       GBP  \
2022-12-30 19:00:00+00:00  0.002751 -0.014591 -0.012273  0.051332 -0.011439   
2022-12-30 20:00:00+00:00  0.002751 -0.014591 -0.012273  0.051332 -0.011439   
2022-12-30 21:00:00+00:00  0.002751 -0.014591 -0.012273  0.051332 -0.011439   
2022-12-30 22:00:00+00:00  0.002751 -0.014591 -0.012273  0.051332 -0.011439   
2022-12-30 23:00:00+00:00  0.002751 -0.014591 -0.012273  0.051332 -0.011439   

                                JPY       NZD       USD  
2022-12-30 19:00:00+00:00  0.004353  0.005516 -0.025649  
2022-12-30 20:00:00+00:00  0.004353  0.005516 -0.025649  
2022-12-30 21:00:00+00:00  0.004353  0.005516 -0.025649  
2022-12-30 22:00:00+00:00  0.004353  0.005516 -0.025649  
2022-12-30 23:00:00+00:00  0.004353  0.005516 -0.025649  


Timestamp('2022-08-01 00:00:00+0000', tz='UTC')

In [103]:
# timedelta(hours=1)

Timestamp('2022-08-01 01:00:00+0000', tz='UTC')

In [68]:
df = raw[raw.index < max]

# min = datetime(2021, 2, 1)
# min = pd.Timestamp(min,tz='UTC')

# df = raw[raw.index > min]

df.shape

(115221, 8)

In [69]:
# Filter requirements.
# T = df.shape[0]         # Sample Period
fs = 30.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
# n = int(T * fs) # total number of samples

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

In [70]:
df = pd.DataFrame(butter_lowpass_filter(df, cutoff, fs, order), index=df.index, columns=df.columns)
split = int(len(df.index)/30)
print(split)
train = df[:-split]
test = df[-split:]
# train.index = pd.to_datetime(train.index).to_period('H').to_timestamp().values
# test.index = pd.to_datetime(test.index).to_period('H').to_timestamp().values
print(f'\n\n{df.shape}\n{type(df.index)}\n{df.index[-1]}\n\n{train.shape}\n{train.index}\n\n{test.shape}\n{test.index}')

3840


(115221, 8)
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
2022-07-31 23:00:00+00:00

(111381, 8)
DatetimeIndex(['2005-01-02 19:00:00+00:00', '2005-01-02 20:00:00+00:00',
               '2005-01-02 21:00:00+00:00', '2005-01-02 22:00:00+00:00',
               '2005-01-02 23:00:00+00:00', '2005-01-03 00:00:00+00:00',
               '2005-01-03 01:00:00+00:00', '2005-01-03 02:00:00+00:00',
               '2005-01-03 03:00:00+00:00', '2005-01-03 04:00:00+00:00',
               ...
               '2021-12-23 10:00:00+00:00', '2021-12-23 11:00:00+00:00',
               '2021-12-23 12:00:00+00:00', '2021-12-23 13:00:00+00:00',
               '2021-12-23 14:00:00+00:00', '2021-12-23 15:00:00+00:00',
               '2021-12-23 16:00:00+00:00', '2021-12-23 17:00:00+00:00',
               '2021-12-23 18:00:00+00:00', '2021-12-23 19:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=111381, freq=None)

(3840, 8)
DatetimeIndex(['2021-12-23 20:00:00+00:00', '2021-12-23 21

## a)  Find p & q for each currency fro the last 10000 values of the trainset

{'AUD': (1, 0, 0), 'CAD': (2, 0, 1), 'CHF': (2, 0, 1), 'EUR': (2, 0, 1), 'GBP': (2, 0, 1), 'JPY': (2, 0, 1), 'NZD': (2, 0, 1), 'USD': (2, 0, 1)}

In [71]:
### Take some optimsed params
# params = {'AUD': (1, 0, 0), 'CAD': (2, 0, 1), 'CHF': (2, 0, 1), 'EUR': (2, 0, 1), 'GBP': (2, 0, 1), 'JPY': (2, 0, 1), 'NZD': (2, 0, 1), 'USD': (2, 0, 1)}

In [ ]:
train_for_params = df[-4320:]

params = {}
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=Warning)
    for c in tqdm(list(train_for_params.columns)):
        res = arma_order_select_ic(train_for_params[c].values, max_ar=4, max_ma=2, ic='aic')
        # print(f'{c} mle retvals: {res.mle_retvals}')
        params[c] = (res.aic_min_order[0], 0, res.aic_min_order[1])
print(params)

In [10]:
%%time
from statsmodels.tsa.stattools import arma_order_select_ic

def optimiseParams(data):

    train_for_params = data[-4320:] ### PARAMS are based on the 1 month of hourly historical data
    params = {}
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=Warning)
        for c in tqdm(list(train_for_params.columns)):
            res = arma_order_select_ic(train_for_params[c].values, max_ar=4, max_ma=2, ic='aic')
            # print(f'{c} mle retvals: {res.mle_retvals}')
            params[c] = (res.aic_min_order[0], 0, res.aic_min_order[1])
    return params

CPU times: user 22 µs, sys: 2 µs, total: 24 µs
Wall time: 27.9 µs


In [12]:
params = optimiseParams(train)
# params = {'AUD': (1, 0, 2), 'CAD': (0, 0, 2), 'CHF': (0, 0, 2), 'EUR': (0, 0, 2), 'GBP': (0, 0, 2), 'JPY': (0, 0, 2), 'NZD': (2, 0, 0), 'USD': (2, 0, 0)}

100%|██████████| 8/8 [03:41<00:00, 27.71s/it]


In [72]:
params

{'AUD': (1, 0, 2),
 'CAD': (0, 0, 2),
 'CHF': (0, 0, 2),
 'EUR': (0, 0, 2),
 'GBP': (0, 0, 2),
 'JPY': (0, 0, 2),
 'NZD': (2, 0, 0),
 'USD': (2, 0, 0)}

# Train on the subset of last rows of the train set

In [73]:
train_df = train[-50000:] 
print(train_df.shape)

# train_df.index = pd.to_datetime(train_df.index)
# train_df.head()

(50000, 8)


In [108]:
train_df.index[-1]

Timestamp('2021-12-23 19:00:00+0000', tz='UTC')

In [133]:
def getArimaGarch(training_subset):
    '''
    Get volatility with ARIMA+GARCH prediction pd.Series for the all training_subset.columns
    '''
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=Warning)
        warnings.warn = lambda *a, **kw: False
            
        arima_forc_dict = {'next_hour_index': []}
        vol_dict = {'next_hour_index': []}
        for column in list(training_subset.columns):

            arima = ARIMA(training_subset[column], order=params[column])
            arima_fit = arima.fit()
            forecast_results = arima_fit.get_forecast(1, alpha=0.05) # get_forecast( ) define the length of predictions
            # let's use train.index to populate indexes with dates
            arima_forc_dict['next_hour_index'] = training_subset.index[-1] + timedelta(hours=1) # hourly indexes we predict
            arima_forc_dict[column] = list(forecast_results.predicted_mean) # forecast values
            print(forecast_results.predicted_mean.index)
            
            # Use residuals in the Garch    
            garch = arch.arch_model(arima_forc_dict[column], p=1, q=1)
            garch_fitted = garch.fit()
            # Use GARCH to predict the residual
            garch_forecast = garch_fitted.forecast(horizon=1,reindex=False)
            predicted_et = garch_forecast.mean['h.1'].iloc[-1]
        
            # Combine both models' output: yt = mu + et
            volatility = arima_forc_dict[column] + predicted_et
            # Put them into dict
            vol_dict['next_hour_index'] = arima_forc_dict['next_hour_index']
            vol_dict[column] = volatility
            # print(colored(f'\n \n Predicted volatility for {column} currency is: {volatility}','red'))   
        
        vol_df = pd.DataFrame.from_dict(vol_dict).set_index('next_hour_index')
        vol_df.index = pd.to_datetime(vol_df.index.values)
        print(colored(f'\n Output at date time {vol_df.index} Dataframe {vol_df}','red'))   
        return vol_df

In [134]:
# %%time
print(train_df.shape)
vol_row_df = getArimaGarch(train_df)

(50000, 8)
RangeIndex(start=50000, stop=50001, step=1)
Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1
RangeIndex(start=50000, stop=50001, step=1)
Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1
RangeIndex(start=50000, stop=50001, step=1)
Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1
RangeIndex(start=50000, stop=50001, step=1)
Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequali

In [135]:
vol_row_df

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
2021-12-23 20:00:00,-0.000015,-0.000105,-0.000107,1.899284e-07,0.000143,0.000101,0.000107,0.000049


# Evaluate ArimaGarch model and return it's results for the `last_hours` from the trainset, save to CSV
function contain split of data inside and embedded window function

In [137]:
def evaluateArimaGarch(data, last_hours=336):
    '''
    Loop getVolatility function over the rows and produce df with predictions
    Create dataframe and file with the all prediction and create a file
    '''    
    ### Split to limit training dataset, 4230 is equal to 6 months depth of hourly data
    split = 2160
    #### Define i for window function and increasing number of rows
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=Warning)
        warnings.warn = lambda *a, **kw: False

        for i in tqdm(range(last_hours)): #### 336 times windows therefore we get predicted hours for the historical dataset
            ### Optional training subset for the getVolatility Arima+Garch model. Train on smaller portion
            train_df = data[-split-i:-last_hours-i]
            # train_df = train_df[-4320:]
            ### Get output for the each row with GetVolatility() using ARIMA+GARCH
            vol_row_df = getArimaGarch(train_df)
            ### Save row to csv file
            file_path = os.path.join(DATA_PATH, "arimagarch")
            Path(file_path).mkdir(parents=True, exist_ok=True)
            vol_row_df.to_csv(os.path.join(file_path, "arimagarch.csv"), index = True, header = False, mode='a')
    return

In [ ]:
## Run global function on the whole subset 
evaluateArimaGarch(df)

# Get Master Volatility and save at once
function contain split of data inside and embedded window function

In [143]:
%%time
from tqdm import tqdm
from pathlib import Path

def getMasterArimaGarch(data, last_hours=336):
        '''
        Loop getVolatility function over the rows and produce df with predictions
        Create dataframe and file with the all prediction and create a file
        '''
               
        ### Split only to limit nr of rows output and time of processing
        split = 2160 # 3 months
        #### Define i for window function and increasing number of rows
        volatilities = {}
        for i in tqdm(range(last_hours)):
            train_df = data[-split-i:-last_hours-i]
            ### Optional training subset for the getVolatility Arima+Garch model. Train on smaller portion
            # train_df = train_df[-4320:]
            ### Get output for the each row with GetVolatility() using ARIMA+GARCH
            vol_row_df = getArimaGarch(train_df)
            volatilities[i] = vol_row_df.values.flatten().tolist()
            ### Increase i for next window for the trainset

        volatilities_df = pd.DataFrame(volatilities).T
        volatilities_df.columns = vol_row_df.columns

        print("\n### CREATING arimagarch.csv ###")
        file_path = os.path.join(DATA_PATH, "arimagarch")
        Path(file_path).mkdir(parents=True, exist_ok=True)
        data.to_csv(os.path.join(file_path, "arimagarch.csv"))

        return volatilities_df

CPU times: user 30 µs, sys: 2 µs, total: 32 µs
Wall time: 36.2 µs


In [ ]:
%%time
results_df = getMasterArimaGarch(df, 336)
results_df

In [39]:
from tqdm import tqdm
from pathlib import Path

def getMasterArimaGarch(self, data):
        '''
        Loop getVolatility function over the rows and produce df with predictions
        Create dataframe and file with the all prediction and create a file
        '''
        
        ### Split only to limit nr of rows output and time of processing
        split = 4320
        test_df = data[-split:]
        #### Define i for window function and increasing number of rows
        i=0
        volatilities = {}
        for row in tqdm(list(test_df.index)):
            train_df = data[-2*split+i:-split+i]
            ### Optional training subset for the getVolatility Arima+Garch model. Train on smaller portion
            # train_df = train_df[-4320:]
            ### Get output for the each row with GetVolatility() using ARIMA+GARCH
            vol_row_df = self.getArimaGarch(train_df)
            volatilities[row] = vol_row_df.values.flatten().tolist()
            ### Increase i for next window for the trainset
            i = i+1

        volatilities_df = pd.DataFrame(volatilities).T
        volatilities_df.columns = vol_row_df.columns

        print("\n### CREATING arimagarch.csv ###")
        file_path = os.path.join(DATA_PATH, "arimagarch")
        Path(file_path).mkdir(parents=True, exist_ok=True)
        data.to_csv(os.path.join(file_path, "arimagarch.csv"))

        return volatilities_df

In [252]:
volatilities_df

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
2022-07-29 07:00:00+00:00,-6.540205e-06,0.000028,-0.000082,-0.000260,-0.000713,-0.002137,-0.002900,-0.005990
2022-07-29 08:00:00+00:00,3.548562e-06,0.000017,0.000006,-0.000002,-0.000117,-0.000440,-0.001049,-0.001857
2022-07-29 09:00:00+00:00,-3.516150e-07,-0.000014,-0.000080,-0.000298,-0.000796,-0.001676,-0.000515,-0.002041
2022-07-29 10:00:00+00:00,-1.161071e-05,0.000037,0.000188,0.000486,0.000700,0.000347,0.000249,-0.000665
2022-07-29 11:00:00+00:00,-1.120008e-05,-0.000032,-0.000126,-0.000617,-0.002158,-0.005259,-0.009179,-0.009749
2022-07-29 12:00:00+00:00,-9.895858e-06,-0.000023,-0.000077,-0.000197,-0.000238,-0.002613,-0.003476,0.000904
2022-07-29 13:00:00+00:00,-1.200998e-05,-0.000029,-0.000175,-0.000791,-0.002628,-0.006549,-0.013112,-0.018245
2022-07-29 14:00:00+00:00,-8.506998e-06,0.000030,0.000124,0.000458,0.000904,0.000626,-0.001325,-0.004629
2022-07-29 15:00:00+00:00,-2.767775e-05,-0.000195,-0.000946,-0.002883,-0.006285,-0.012426,-0.015304,-0.020481
2022-07-29 16:00:00+00:00,-8.760380e-06,-0.000064,-0.000206,-0.000389,-0.003349,0.000646,0.002323,0.008482


In [143]:
### What's the actual value?
vol_actual = pd.DataFrame(test.iloc[0] - train_df.iloc[-1]).T
vol_actual['next_hour_index'] = test.index[0]
vol_actual = vol_actual.set_index('next_hour_index')
vol_actual

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
next_hour_index,,,,,,,,
2021-12-23 20:00:00+00:00,-0.000002,-0.000015,-0.000055,-0.00012,-0.00015,-0.000037,0.000302,0.00086


In [126]:
### what's the performacne of the prediction in relation to the actual values?
# perfomance = vol_predicted - vol_actual
perf = pd.DataFrame(vol_df-test.iloc[0]).reset_index(drop=True)   #, columns=vol_df.columns).set_index=test.iloc[0]
perf['next_hour_index'] = test.index[0]
perf.set_index('next_hour_index')

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
next_hour_index,,,,,,,,
2021-12-23 20:00:00+00:00,-0.000008,0.000007,0.000025,0.000021,0.00002,-0.000039,-0.000207,-0.000504


# Test to save one huge dataset

In [ ]:
# Loop over all the test values
volatilities = {'next_hour_index': []}

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=Warning)
    for row in list(test.index): 
        arima_forc_dict = {'next_hour_index': []}
        
        volatilities['next_hour_index'] = row
        
        vol_dict = {'next_hour_index': []}
        for column in list(train_for_params.columns):

            arima = ARIMA(train_for_params[column], order=params[column])
            arima_fit = arima.fit()
            forecast_results = arima_fit.get_forecast(1, alpha=0.05) # get_forecast( ) define the length of predictions
            
            arima_forc_dict['next_hour_index'] = list(forecast_results.predicted_mean.index) # hourly indexes we predict
            arima_forc_dict[column] = list(forecast_results.predicted_mean) # forecast values
            
            # Use residuals in the Garch    
            garch = arch.arch_model(arima_forc_dict[column], p=1, q=1)
            garch_fitted = garch.fit()
            # Use GARCH to predict the residual
            garch_forecast = garch_fitted.forecast(horizon=1,reindex=False)
            predicted_et = garch_forecast.mean['h.1'].iloc[-1]
        
            # Combine both models' output: yt = mu + et
            volatility = arima_forc_dict[column] + predicted_et
            # Put them into dict
            vol_dict['next_hour_index'] = list(forecast_results.predicted_mean.index)
            vol_dict[column] = volatility
            print(colored(f'\n \n Predicted volatility for {column} currency is: {volatility}','red'))   
            
            # One row of prediction in dict format
            volatilities[column] = [volatility]